In [ ]:
from datetime import datetime
from uuid import uuid4
from dateutil.tz import tzlocal
import numpy as np
import scipy.io
import os, yaml, glob, json
from nwbwidgets import nwb2widget
from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.behavior import Position, SpatialSeries
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject

In [ ]:
def read_names(filename):
    assignment  = filename.split('.')[0].split('-')[1]
    number      = filename.split('.')[0].split('-')[2]
    return np.asarray([assignment, number])

def create_nwb(config, path):

    SessionDate = path.split('/')[-1].split('.')[-2].split('_')[0]
    SessionTime = path.split('/')[-1].split('.')[-2].split('_')[1]
    SubjectName = path.split('/')[-1].split('.')[1].split('_')[1]
    date_format = "%Y%m%d %H%M%S"

    if config['subject']['subject_id'].lower() != SubjectName.lower():
        raise ValueError("Subject Name incorrect.")
    
    session_start_time = datetime.strptime(SessionDate+' '+SessionTime, date_format)

    ################ CREATE NWB FILE WITH METADATA ################################
    ###############################################################################
    nwbfile = NWBFile(
        session_description     = config['session_info']['session_description'],
        identifier              = config['metadata']['identifier'],
        session_start_time      = session_start_time,
        file_create_date        = config['metadata']['file_create_date'],
        experimenter            = config['general']['lab_info']['experimenter'],
        experiment_description  = config['general']['experiment_info']['experiment_description'],
        session_id              = config['session_info']['session_id'],
        lab                     = config['general']['lab_info']['lab'],                     
        institution             = config['general']['lab_info']['institution'],                                    
        keywords                = config['general']['experiment_info']['keywords'],
        protocol                = config['general']['experiment_info']['protocol'],
        related_publications    = config['general']['experiment_info']['related_publications'],
        surgery                 = config['general']['experiment_info']['surgery']
    )

    ################ CREATE SUBJECT ################################################
    ################################################################################
    nwbfile.subject = Subject(
        subject_id  = config['subject']['subject_id'],
        date_of_birth= config['subject']['date_of_birth'],
        species     = config['subject']['species'],
        sex         = config['subject']['sex'],
    )

    ################ CREATE HARDWARE LINKS #########################################
    ################################################################################
    nwbfile.create_device(
        name        = config['hardware']['system_name'], 
        description = config['hardware']['system_description'], 
        manufacturer= config['hardware']['system_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['adapter_manuf'], 
        description = config['hardware']['adapter_description'], 
        manufacturer= config['hardware']['adapter_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['monitor_name'], 
        description = config['hardware']['monitor_description'], 
        manufacturer= config['hardware']['monitor_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['photodiode_name'], 
        description = config['hardware']['photodiode_description'], 
        manufacturer= config['hardware']['photodiode_manuf']
    )
    
    nwbfile.create_device(
        name        = 'Software Used', 
        description = str(['Mworks Client: '+config['software']['mwclient_version'],\
                        'Mworks Server: '+config['software']['mwserver_version'],\
                        'OS: '+config['software']['OS'],\
                        'Intan :'+config['software']['intan_version']])
    )

    ################ CREATE ELECTRODE LINKS ########################################
    ################################################################################
    electrodes = nwbfile.create_device(
        name        = config['hardware']['electrode_name'], 
        description = config['hardware']['electrode_description'], 
        manufacturer= config['hardware']['electrode_manuf']
    )

    all_files = sorted(os.listdir(os.path.join(path, 'SpikeTimes')))
    
    name_accumulator = []
    for file in all_files:
        name_accumulator.append(read_names(file))
    names = np.vstack(name_accumulator)

    nwbfile.add_electrode_column(name="label", description="label of electrode")
    groups, count_groups = np.unique(names[:,0], return_counts =True)
    ids                  = names[:,1]
    counter              = 0
    # create ElectrodeGroups A, B, C, ..
    for group, count_group in zip(groups, count_groups):
        electrode_group = nwbfile.create_electrode_group(
            name        = "group_{}".format(group),
            description = "Serialnumber: {}. Adapter Version: {}".format(config['array_info']['array_{}'.format(group)]['serialnumber'],\
                            config['array_info']['array_{}'.format(group)]['adapterversion']),
            device      = electrodes,
            location    = 'hemisphere, region, subregion: '+str([config['array_info']['array_{}'.format(group)]['hemisphere'],\
                                config['array_info']['array_{}'.format(group)]['region'],
                                config['array_info']['array_{}'.format(group)]['subregion']]),
            position    = config['array_info']['array_{}'.format(group)]['position']
        )

        # create Electrodes 001, 002, ..., 032 in ElectrodeGroups per channel
        for ichannel in range(count_group):
            nwbfile.add_electrode(
                group       = electrode_group,
                label       = ids[counter],
                location    = 'row, col, elec'+str(json.loads(config['array_info']['intan_electrode_labeling_[row,col,id]'])[counter])
            )
            counter += 1        

    return nwbfile



In [ ]:
path = '/om/user/aliya277/inventory/norm_FOSS.sub_pico.20230823_124104.proc'
config_path = '/om/user/aliya277/dandi_brainscore'
with open(os.path.join(config_path,"config_nwb.yaml") , "r") as f:
        config = yaml.load(f, Loader = yaml.FullLoader)

nwbfile = create_nwb(config,path)
nwbfile

/home/aliya277/anaconda3/envs/dandibs/lib/python3.10/site-packages/pynwb/file.py:471: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


root pynwb.file.NWBFile at 0x47390306466832
Fields:
  devices: {
    DET36A2 Biased Si Detector <class 'pynwb.device.Device'>,
    Electrode <class 'pynwb.device.Device'>,
    LG UltraGear <class 'pynwb.device.Device'>,
    RecordingSystem <class 'pynwb.device.Device'>,
    Ripple Neuro <class 'pynwb.device.Device'>,
    Software Used <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    group_A <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_B <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_C <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_D <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_E <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_F <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: Task: Rapid serial visual presentation (RSVP). Trial: Try 8 images - x ms on, x ms grey buch check mworks file for specifics. Reward:s juice reward after every trial. Imageset: x Images of x
  experimenter: ['Goulding, Sarah']
  file_create_date: [datetime.datetime(2023, 9, 12, 14, 9, 19, 670169, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000)))]
  identifier: 85bbbacd-9d9f-45f3-b57e-1e9674a22bcc
  institution: McGovern Institute for Brain Research
  keywords: ['Vistual Stimuli' 'Object Recognition' 'Inferior temporal cortex (IT)'
 'Ventral visual pathway']
  lab: DiCarlo
  protocol: Rapid serial visual presentation (RSVP)
  related_publications: ['']
  session_description: Number of Repetitions: x
  session_id: x
  session_start_time: 2023-08-23 12:41:04-04:00
  subject: subject pynwb.file.Subject at 0x47390306463568
Fields:
  age__reference: birth
  date_of_birth: 2019-12-04 00:00:00-05:00
  sex: male
  species: Macaca mulatta
  subject_id: pico

  surgery: 3x Utah Array Implant + Headpost
  timestamps_reference_time: 2023-08-23 12:41:04-04:00

In [ ]:
for filename in os.listdir(os.path.join(path, 'SpikeTimes')):
    if filename.endswith('.mat'):
        file_path = os.path.join(path, 'SpikeTimes', filename)
        data = scipy.io.loadmat(file_path)
        nwbfile.add_unit(spike_times=data['spike_time_ms'].flatten())


In [ ]:
from nwbwidgets import nwb2widget

nwb2widget(nwbfile)